# PETCA - Projeto de Análise de Contas de Energia com Aprendizado de Máquina e Redes Neurais

## Índice
- [Modelos Utilizados](#modelos-utilizados)
- [Importando Pacotes e Bibliotecas](#importando-os-pacotes-e-bibliotecas)
- [Importando os Datasets](#importando-os-datasets)
- [Análise Inicial dos Datasets](#análise-inicial-dos-datasets)
- [Análise Exploratória dos Dados](#aed)
- [Criando os Modelos](#criando-os-modelos)
- [Treinando os Modelos](#treinando-os-modelos)
- [Resultados os Modelos](#resultados-dos-modelos)
    - [Realização dos Testes](#testes)
    - [Qualidade dos Modelos](#qualidade-dos-testes-e-resultados)
- [Discussão](#discussão)

## Modelos Utilizados
- Árvore de Decisão (Decision Tree)
- Ensemble
- Floresta Randômica (Random Forest)
- Redes Neurais Convolucionais
- Regressão Linear
- Regressão Polinomial
- Support Vector Machine (SVM)

## Importando os pacotes e bibliotecas

In [ ]:
# biblioteca para realizar o corte teste | treino
from sklearn.model_selection import train_test_split

# bibliotecas de classificacao
## Floresta Randomica;
## Arvore de Decisao; e
## Support Vector Machine (SVM).
from sklearn.ensemble  import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# biblioteca de modelos polinomiais
from sklearn.preprocessing import PolynomialFeatures

# biblioteca de modelos lineares
## Regressao Linear; e
## Support Vector Machine (SVM).
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# bibliotecas de suporte -----
## Impressao de Graficos
from matplotlib import pyplot as plt
import seaborn as sns

## Bibliotecas Base
import pandas as pd
import numpy as np
# ----------------------------

# bibliotecas e pacotes do TensorFlow
## Redes Neurais Convolucionais
import tensorflow as tf
from keras import layers, models

## Importando os datasets

In [ ]:
df_residencial_raw = pd.read_csv("./databases/raw/CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE - CONSUMO COMERCIAL POR UF.csv", sep = ",", index_col = 0)

## Análise Inicial dos Datasets

### Consumo Residencial por UF

In [ ]:
df_residencial_raw.sample(10)

#### Quantidade de valores nulos

In [ ]:
df_residencial_transposto = df_residencial_raw.transpose()
df_residencial_transposto.sample(10)

In [ ]:
columns_to_be_droped = [
    "Consumo de energia elétrica na rede (MWh)",
    "Sistema SIMPLES",
    "Nota: atualização defasada para não antecipar informações de distribuidoras que devem obedecer às intruções da CVM sobre publicação de resultados.",
    "TOTAL POR UF"
]

In [ ]:
df_residencial_transposto.drop(columns = columns_to_be_droped, inplace = True, axis = "columns")
df_residencial_transposto.sample(10)

## Renomeando colunas que não possuiam nomes

In [ ]:
columns_to_rename = pd.Series(df_residencial_transposto.columns)
columns_to_rename = columns_to_rename.fillna("new_name" + (columns_to_rename.groupby(columns_to_rename.isnull()).cumcount() + 1).astype(str))
df_residencial_transposto.columns = columns_to_rename
df_residencial_transposto.sample(10)

### Excluindo coluna insignificante

In [ ]:
df_residencial_transposto.drop(columns = ["new_name1"], inplace = True, axis = "columns")
df_residencial_transposto.sample(10)

### Renomeando colunas "ano" e "mês"

In [ ]:
columns_to_rename = {
    "new_name2" : "ano",
    "new_name3" : "mes"
}

df_residencial_transposto.rename(mapper = columns_to_rename, axis = "columns", inplace = True)
df_residencial_transposto.head(5)

### Preenchendo a coluna "ano" com valores corretos do ano

In [ ]:
df_residencial_transposto["ano"] = df_residencial_transposto["ano"].ffill()

## Transformando colunas importantes em Índice

In [ ]:
df_residencial_transposto["ano"].unique()

In [ ]:
df_residencial_transposto["mes"].unique()

In [ ]:
df_residencial_transposto.columns

In [ ]:
df_residencial_transposto.reset_index(inplace = True)
df_residencial_transposto["ano"] = df_residencial_transposto["ano"].astype("str")
df_residencial_transposto["mes"] = df_residencial_transposto["mes"].astype("str")
df_residencial_transposto.set_index(["ano", "mes"], inplace = True)
df_residencial_transposto.drop(columns = ["index"], axis = "columns", inplace = True)

In [ ]:
df_residencial_transposto.sample(10)

### Deletando a linha NaN do índice

In [ ]:
df_residencial_processed = df_residencial_transposto.iloc[ : df_residencial_transposto.shape[0] - 12, :]
df_residencial_processed.tail(20)

### Verificandos os tipos das colunas do dataset

In [ ]:
df_residencial_processed.dtypes

### Trocando os tipos dos dados para melhor compressão e compreensão do dataset

Os dados serão trocados para o tipo "unsigned integer" de 32 bits

In [ ]:
for coluna in df_residencial_processed.columns:
    df_residencial_processed[coluna] = [valor.replace(".", "") for valor in df_residencial_processed[coluna]]

In [ ]:
df_residencial_processed["Rondônia"] = df_residencial_processed["Rondônia"].astype("uint32")
df_residencial_processed["Acre"] = df_residencial_processed["Acre"].astype("uint32")
df_residencial_processed["Amazonas"] = df_residencial_processed["Amazonas"].astype("uint32")
df_residencial_processed["Roraima"] = df_residencial_processed["Roraima"].astype("uint32")
df_residencial_processed["Pará"] = df_residencial_processed["Pará"].astype("uint32")
df_residencial_processed["Amapá"] = df_residencial_processed["Amapá"].astype("uint32")
df_residencial_processed["Tocantins"] = df_residencial_processed["Tocantins"].astype("uint32")
df_residencial_processed["Maranhão"] = df_residencial_processed["Maranhão"].astype("uint32")
df_residencial_processed["Piauí"] = df_residencial_processed["Piauí"].astype("uint32")
df_residencial_processed["Ceará"] = df_residencial_processed["Ceará"].astype("uint32")
df_residencial_processed["Rio Grande do Norte"] = df_residencial_processed["Rio Grande do Norte"].astype("uint32")
df_residencial_processed["Paraíba"] = df_residencial_processed["Paraíba"].astype("uint32")
df_residencial_processed["Pernambuco"] = df_residencial_processed["Pernambuco"].astype("uint32")
df_residencial_processed["Alagoas"] = df_residencial_processed["Alagoas"].astype("uint32")
df_residencial_processed["Sergipe"] = df_residencial_processed["Sergipe"].astype("uint32")
df_residencial_processed["Bahia"] = df_residencial_processed["Bahia"].astype("uint32")
df_residencial_processed["Minas Gerais"] = df_residencial_processed["Minas Gerais"].astype("uint32")
df_residencial_processed["Espírito Santo"] = df_residencial_processed["Espírito Santo"].astype("uint32")
df_residencial_processed["Rio de Janeiro"] = df_residencial_processed["Rio de Janeiro"].astype("uint32")
df_residencial_processed["São Paulo"] = df_residencial_processed["São Paulo"].astype("uint32")
df_residencial_processed["Paraná"] = df_residencial_processed["Paraná"].astype("uint32")
df_residencial_processed["Santa Catarina"] = df_residencial_processed["Santa Catarina"].astype("uint32")
df_residencial_processed["Rio Grande do Sul"] = df_residencial_processed["Rio Grande do Sul"].astype("uint32")
df_residencial_processed["Mato Grosso do Sul"] = df_residencial_processed["Mato Grosso do Sul"].astype("uint32")
df_residencial_processed["Mato Grosso"] = df_residencial_processed["Mato Grosso"].astype("uint32")
df_residencial_processed["Goiás"] = df_residencial_processed["Goiás"].astype("uint32")
df_residencial_processed["Distrito Federal"] = df_residencial_processed["Distrito Federal"].astype("uint32")

### Conferindo a troca de tipos

In [ ]:
df_residencial_processed.dtypes

## Salvando os datasets tratados
### Eles serão salvos em .pkl (pickle)
Essa extensão criada pelos desenvolvedores da biblioteca pandas salva os metadados dos datasets enquanto comprime seu tamanho de arquivo

In [ ]:
df_residencial_processed.to_pickle(path = "./databases/processed/classes-consumoComercialPorUF.pkl")

## AED
### Análise Exploratória dos Dados

### Lendo os datasets tratados em formato pickle

In [ ]:
df_residencial_processed = pd.read_pickle("./databases/processed/classes-consumoComercialPorUF.pkl")

### Consumo Residencial por UF

#### Informações básicas aglutinadas

In [ ]:
df_residencial_processed.info()

#### Valores demonstrados de forma numéricas

In [ ]:
for coluna in df_residencial_processed.columns:
    print("Estado " + coluna, end = "\n")
    print(f"Valor mínimo: {df_residencial_processed[coluna].min()}", end = "\n")
    print(f"Valor máximo: {df_residencial_processed[coluna].mean()}", end = "\n")
    print(f"Média: {df_residencial_processed[coluna].mean()}", end = "\n")
    print(f"Mediana: {df_residencial_processed[coluna].median()}", end = "\n")
    print(f"Desvio padrão: {df_residencial_processed[coluna].std()}", end = "\n\n")
    print(f"Moda: {df_residencial_processed[coluna].mode()}", end = "\n")

#### Valores demonstrados de forma gráfica

In [ ]:
figura, eixos = plt.subplots(nrows = 5, ncols = 1, sharex = True, figsize = (14, 12))
plt.suptitle("Consumo por estados - em Kw/h")

sns.barplot(df_residencial_processed.mean(), ax = eixos[0])
eixos[0].set_title("Média")

sns.barplot(df_residencial_processed.median(), ax = eixos[1])
eixos[1].set_title("Mediana")

sns.barplot(df_residencial_processed.max(), ax = eixos[2])
eixos[2].set_title("Valor Máximo")

sns.barplot(df_residencial_processed.min(), ax = eixos[3])
eixos[3].set_title("Valor Mínimo")

sns.barplot(df_residencial_processed.std(), ax = eixos[4])
eixos[4].set_title("Desvio Padrão")

plt.xticks(rotation = 45, ha = "right")
plt.show()

In [ ]:
figura = plt.figure(figsize = (20, 8))

sns.boxplot(df_residencial_processed, orient = "v")
plt.xticks(rotation = 45, ha = "right")
plt.title("Consumo por estados - em Kw/h")

plt.show()

## Criando os Modelos

### Criando uma nova coluna com valores categóricos
- Alto
- Baixo
- Médio

Os valores servirão para a previsão do modelo de árvore de decisão

In [ ]:
colunas = df_residencial_processed.columns
consumo_total = []

for coluna in colunas:
    consumo_do_estado_atual = df_residencial_processed.T.loc[coluna, :].sum()
    # print("Coluna: ", coluna, end = " ")
    # print(consumo_do_estado_atual, end = "\n\n")
    consumo_total.append(consumo_do_estado_atual)
    
# print(consumo_total)

In [24]:
consumo_total = pd.Series(consumo_total)
primeiro_quartil = np.quantile(consumo_total, 0.25)
terceiro_quartil = np.quantile(consumo_total, 0.75)

print("Mediana: ", consumo_total.median())
print("Primeiro quartil: ", primeiro_quartil)
print("Terceiro quartil: ", terceiro_quartil)

28334063.0
Primeiro quartil:  12484022.0
Terceiro quartil:  55802407.5


In [25]:
for i in consumo_total.index:
    if consumo_total[i] > primeiro_quartil and consumo_total[i] < terceiro_quartil:
        consumo_total[i] = 0 # NORMAL
    elif consumo_total[i] < primeiro_quartil:
        consumo_total[i] = 1 # BAIXO
    else:
        consumo_total[i] = 2 # ALTO
        
consumo_total

0     1
1     1
2     0
3     1
4     0
5     1
6     1
7     0
8     1
9     0
10    0
11    0
12    0
13    0
14    1
15    2
16    2
17    0
18    2
19    2
20    2
21    2
22    2
23    0
24    0
25    0
26    0
dtype: uint64

In [27]:
df_residencial_com_valor_categorico = df_residencial_processed.T
df_residencial_com_valor_categorico["consumo"] = consumo_total
df_residencial_com_valor_categorico.head(10)

ano                                    2004                              \
mes                                     JAN    FEV    MAR    ABR    MAI   
Empresa de Pesquisa Energética - EPE                                      
Rondônia                              25870  23367  24153  24113  25789   
Acre                                   7895   7329   7420   7345   7142   
Amazonas                              49832  50457  47374  49875  50280   
Roraima                                6141   5822   5494   6035   5548   
Pará                                  78075  72467  75857  78779  79714   
Amapá                                 12164   3894   8639   8461   7704   
Tocantins                             13832  12160  13819  14883  15465   
Maranhão                              40478  37893  40878  40503  42202   
Piauí                                 25398  20769  20862  24450  22877   
Ceará                                 95718  86036  90007  97130  90758   

ano                                                                      ...  \
mes                                     JUN    JUL    AGO    SET    OUT  ...   
Empresa de Pesquisa Energética - EPE                                     ...   
Rondônia                              24112  24312  25604  25980  27250  ...   
Acre                                   6955   7205   7580   8309   7910  ...   
Amazonas                              50734  53678  52589  55148  55374  ...   
Roraima                                5472   5876   5820   5652   5980  ...   
Pará                                  82470  79919  83589  83100  82032  ...   
Amapá                                  8309   8733   7858   8850   9621  ...   
Tocantins                             16111  14775  15815  17041  16450  ...   
Maranhão                              42065  41961  43550  44972  43339  ...   
Piauí                                 24519  22937  22760  25997  25217  ...   
Ceará                                 93769  87315  92398  97622  93272  ...   

ano                                     2023                                  \
mes                                      ABR     MAI     JUN     JUL     AGO   
Empresa de Pesquisa Energética - EPE                                           
Rondônia                               59678   63534   63088   60077   65204   
Acre                                   21441   22452   21567   22285   24295   
Amazonas                              123118  133650  132674  138483  148584   
Roraima                                23108   23453   22501   22241   26455   
Pará                                  184017  189880  197274  198387  206946   
Amapá                                  22833   23913   25023   25092   30217   
Tocantins                              37873   40181   41039   40260   41552   
Maranhão                               93987  104726  103675  105214  109107   
Piauí                                  69513   74618   73832   74616   79158   
Ceará                                 198290  193551  209262  187632  201380   

ano                                                                  consumo  
mes                                      SET     OUT     NOV     DEZ          
Empresa de Pesquisa Energética - EPE                                          
Rondônia                               68834   67340   69317   68373     NaN  
Acre                                   25331   25390   25277   24610     NaN  
Amazonas                              147337  146305  140994  133602     NaN  
Roraima                                26748   26993   26314   24974     NaN  
Pará                                  207992  213696  218560  206848     NaN  
Amapá                                  25102   27594   27976   27175     NaN  
Tocantins                              43376   44697   43095   43730     NaN  
Maranhão                              108778  113651  111837  111114     NaN  
Piauí                                  84283   85288   84302   81191     

### Consumo Residencial por UF

In [ ]:
from models import DecisionTree

arvore_decisao = DecisionTree()

## Treinando os Modelos

## Resultados dos Modelos

### Testes

### Qualidade dos Testes e Resultados

## Discussão

A discussão vai aqui